In [1]:
%load_ext autoreload
%autoreload 2
import multiprocessing as mp

from output_check2 import output_check
from synthpop.recipes.starter6 import Starter
starter_ver='6'

from synthpop.synthesizer import synthesize_all, enable_logging 
import pandas as pd
import os,time,datetime
import pickle

import gc



outputfolder='/home/da/tools/synthpop/demos/outputs/'

# Synthesize by county 

In [2]:
#for whole county
time0=time.time()
print str( datetime.datetime.now())
results=[]

#synthesize_counties(["Livingston County","Oakland County","Washtenaw County","Wayne County","Monroe County","Macomb County","St. Clair County"])
if __name__=='__main__':
    counties=["Macomb County"]
    finalcheck=[]
    for county in counties:
        phases=3
        df_check=None
        for phase in range(phases):
            print "\n county = ",county," phase = ", phase, "\n"
            
            pathout=outputfolder+county+"_starter%s_"%(starter_ver) #output folder
            
            starter = Starter(os.environ["CENSUS"], "MI", county, phase=phase)
           
            households, persons, fit, hh_marginal, pp_marginal = synthesize_all(starter)

            #households.index.name='hh_id' #result[0] is hh
            #households['serialno'] = households['serialno'].map(lambda x: '%f' % x)
            #persons['serialno'] = persons['serialno'].map(lambda x: '%f' % x)
            #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, not use scentific, pplies to all fp numbers 
            #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change output format for serialno only
            
            #HHs and Persons output
            if phase == phases-1:
                households.to_csv(pathout+'_households.csv')
                persons.to_csv(pathout+'_persons.csv')


            ##process check tables
            dict_check=output_check({'households':[hh_marginal,households], 'persons':[pp_marginal,persons]})
            #print dict_check.keys() #['households', 'persons', 'hhsize', 'person_factor', 'hhsize7plus']

            outfiles=[pathout+"_check_households.csv",pathout+"_check_households_bycat.csv", \
                      pathout+"_check_persons.csv",pathout+"_check_persons_bycat.csv", \
                      pathout+"_check_hhsize.csv"]

            #set index to check tables and rename columns with phase #
            dfcheck=[dict_check['households'][0],dict_check['households'][1], \
                     dict_check['persons'][0],dict_check['persons'][1], \
                     dict_check['hhsize']]
            indx=['state','county','tract','block group','cat_name']
            indxl=[indx, indx+['cat_value']]*2+[indx+['hh_size']]

            for ind, df in zip(indxl, dfcheck):
                df.set_index(ind, inplace=True)
                df.rename(columns={col:col+"_"+str(phase) for col in df.columns}, inplace=True)

            #join check table from multiple runs            
            if len(finalcheck)==0:
                finalcheck=dfcheck
            else:
                for i in range(5):
                    finalcheck[i]=pd.merge(finalcheck[i], dfcheck[i], left_index=True, right_index=True, how='left')

            #create additional id fields for analysis purpose  
            if phase == phases-1:
                for i in range(5):
                    finalcheck[i].reset_index(inplace=True)
                    finalcheck[i]['bg10']=finalcheck[i]['tract']+finalcheck[i]['block group']
                    finalcheck[i]['geoid10']=finalcheck[i]['state']+finalcheck[i]['county']+ \
                                                finalcheck[i]['tract']+finalcheck[i]['block group']  
                    finalcheck[i].to_csv(outfiles[i])

            if phase == 0:
                dict_check['hhsize7plus'].to_csv("hh7_persons_drawing.csv")
            if phase == 1:
                dict_check['person_factor'].to_csv("hh_size_person_factor.csv")
            if phase == 2:
                os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
                os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")

            households,persons,marginals,dict_check = None, None, None, None
            

            with open(pathout+'_fit_stats.pkl', 'wb') as f:
                pickle.dump(fit, f)

            del households,persons,hh_marginal,pp_marginal,dict_check, dfcheck
            
            gc.collect()  
            
print 'total time', time.time()-time0
print str( datetime.datetime.now())

2016-04-26 14:45:53.062061

 county =  Macomb County  phase =  0 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
memory 1 131952640
multiprocessing time in seconds:  865.428035021
memory multiprocessing end 3125903360
result produce time in seconds: 

/home/da/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
output_check2.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_marg_size['persons']=sum_size
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constrai

 874.185871124
memory result end 6424207360

 county =  Macomb County  phase =  1 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
memory 1 3589496832


/home/da/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2705: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
output_check2.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfhh7size['hh_size_weight_7']=dfhh7size['persons']/dfhh7size['households']


multiprocessing time in seconds:  828.340661049
memory multiprocessing end 6395564032
result produce time in seconds:  837.33310008
memory result end 9196371968

 county =  Macomb County  phase =  2 

Synthesizing at geog level: 'block_group' (number of geographies is 627)
memory 1 6580948992


/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.in

multiprocessing time in seconds:  821.637382984
memory multiprocessing end 9287647232
result produce time in seconds:  830.649375916
memory result end 11986436096
total time 2722.57479
2016-04-26 15:31:15.636865


/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.index)
/home/da/anaconda2/lib/python2.7/site-packages/SynthPop-0.1.dev0-py2.7.egg/synthpop/draw.py:157: FutureWarning: diff is deprecated. Use difference instead
  diff = constraints.index.diff(counts.in

# Synthesize by selected Block Groups

In [ ]:
###for selected BGs
#enable_logging()
time0=time.time()
geoname="problembgs3.csv"
pathout=outputfolder+geoname+"_starter%s_"%(starter_ver)

dfbg=pd.read_csv(geoname,index_col=None,dtype=str) #make sure csv column in text not number, so read as'099' not 99
indd=[ind for i, ind in dfbg.iterrows()]
phases=3
finalcheck=[]
for phase in range(phases):
    print "\nphase = ", phase, "\n"
    starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County', phase=phase)
    households, persons, fit, hh_marginal, pp_marginal = synthesize_all(starter, indexes=indd)
    
    #HHs and Persons output
    if phase == phases-1:
        households.to_csv(pathout+'_households.csv')
        persons.to_csv(pathout+'_persons.csv')
 
    ##process check tables
    dict_check=output_check({'households':[hh_marginal,households], 'persons':[pp_marginal,persons]})
    #print dict_check.keys() #['households', 'persons', 'hhsize', 'person_factor', 'hhsize7plus']
    
    outfiles=[pathout+"_check_households.csv",pathout+"_check_households_bycat.csv", \
              pathout+"_check_persons.csv",pathout+"_check_persons_bycat.csv", \
              pathout+"_check_hhsize.csv"]

    #set index to check tables and rename columns with phase #
    dfcheck=[dict_check['households'][0],dict_check['households'][1], \
             dict_check['persons'][0],dict_check['persons'][1], \
             dict_check['hhsize']]
    indx=['state','county','tract','block group','cat_name']
    indxl=[indx, indx+['cat_value']]*2+[indx+['hh_size']]

    for ind, df in zip(indxl, dfcheck):
        df.set_index(ind, inplace=True)
        df.rename(columns={col:col+"_"+str(phase) for col in df.columns}, inplace=True)

    #join check table from multiple runs            
    if len(finalcheck)==0:
        finalcheck=dfcheck
    else:
        for i in range(5):
            finalcheck[i]=pd.merge(finalcheck[i], dfcheck[i], left_index=True, right_index=True, how='left')
    
    #create additional id fields for analysis purpose  
    if phase == phases-1:
        for i in range(5):
            finalcheck[i].reset_index(inplace=True)
            finalcheck[i]['bg10']=finalcheck[i]['tract']+finalcheck[i]['block group']
            finalcheck[i]['geoid10']=finalcheck[i]['state']+finalcheck[i]['county']+ \
                                        finalcheck[i]['tract']+finalcheck[i]['block group']  
            finalcheck[i].to_csv(outfiles[i])

    if phase == 0:
        dict_check['hhsize7plus'].to_csv("hh7_persons_drawing.csv")
    if phase == 1:
        dict_check['person_factor'].to_csv("hh_size_person_factor.csv")
    if phase == 2:
        os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
        os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")
        
    households,persons,marginals,dict_check = None, None, None, None
    
print time.time()-time0